In [1]:
%matplotlib inline

In [2]:
import numpy as np

import pprint
import time
import torch
import torchvision

import matplotlib.pyplot as plt
import torch.nn.functional as F

/opt/conda/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


We're going to work on matrix multiplication today.
Matrices and multi-dimensional arrays are essential to scientific computing and to neural networks in particular.
Neural networks, for the most part, are just a series of matrix multiplications and vector operations.
So, it's essential that you learn about these!

The primary framework for matrix operations is the BLAS (basic linear algebra subprograms) and these are the actual implementations used in scientific computing.
Today, we're going to build our own:
- vector class
- matrix class
- **dot product**
- **saxpy** function (level 1 BLAS)
- **gaxpy** function (level 2 BLAS)
- **level 3** BLAS function

We're then going to work on understanding:
- the `ndarray` class in numpy
- the `tensor` class in pytorch
- the stride method in `tensor`

After this, we're going to move onto working with these in the numpy and pytorch libraries.
We'll see that numpy is faster than our implementations (they are heavily optimized by thousands of researchers and developers over many years).
It's still important to understand what is happening under the hood!

Let's look at vectors first!
A vector is just a sequence of numbers.
For example:
$$
(1, 3, 5)
$$
is a 3-dimensional vector!
It can be arbitrarily long, so we can have a $1,000,000$-dimensional vector.

We've already seen a python class that works just like a vector.
Do you know what it is?

Let's use these to build our vector class.

In [3]:
class Vector():
    
    def __init__(self, values):
        # This should have an input.
        # What do we need to input to create a vector?
        self.num_values = len(values)
        self.values = tuple(values)
    
    def __getitem__(self, idx):
        return self.values[idx]
    
    def __len__(self):
        return len(self.values)
    
    def __add__(self, other):
        if self.num_values != other.num_values:
            raise Exception(f"Dimension mismatch: {self.num_values} != {other.num_values}")
            
        add = []
        for num in range(self.num_values):
            add.append(self.values[num] + other.values[num])
        return Vector(add)
    
    def __repr__(self):
        return f"Vector({self.values})"
    
    def __str__(self):
        vec_string = ''
        for num in range(self.num_values):
            vec_string = vec_string + str(self.values[num])
        return vec_string

One reason we might want to build our own class rather than use the python classes that look like vectors is readability.
Tuples and lists are general classes and can be used for anything.
Whereas if we create a Vector class, then that tells anyone who is working on our code that these represents vectors.
This means that they wouldn't do something that isn't allowed in vector operations.
For example, they would know our `Vector` class should not have an `append` method, like `list` has.

We're going to talk about the dot product function and then try to implement it.
If we have two vectors of the same dimension, $\textbf{x} = (x_{1}, \ldots, x_{n})$ and $\textbf{y} = (y_{1}, \ldots, y_{n})$, then
$$
\textbf{x} \cdot \textbf{y} = x_{1}y_{1} + x_{2}y_{2} + \cdots + x_{n}y_{n} = \sum_{i = 1}^{n} x_{i}y_{i}.
$$
We just multiply the elements together and then sum them!
This gives us a chance to practice our `for` loops!

In [4]:
x = Vector((1, 2, 3))
x[1]

2

In [5]:
def dot_product(vector1, vector2):
    # We need to initialize a value
    dot = 0
    
    # We need to do a for loop
    for num in range(vector1.num_values):
        dot += vector1[num] * vector2[num]
        
    return dot

In [6]:
x = Vector((1, 3, 5))
y = Vector((2, 4, 6))

print(dot_product(x, y))

44


Another bit of `for` loops practice is programming `saxpy`.
What `saxpy` stands for is single precision $a$ times $\textbf{x}$ plus $\textbf{y}$.

The inputs are a scalar $a$ and two vectors of the same dimension: $\textbf{x}$ and $\textbf{y}$.
The output should be a vector.
The implementation is:
$$
a \textbf{x} + \textbf{y} = a \cdot(x_{1}, \ldots, x_{n}) + (y_{1}, \ldots, y_{n}) = (ax_{1} + y_{1}, \ldots, ax_{n} + y_{n})
$$
Let's implement this!

In [7]:
def saxpy(a, x, y):
    # a: a float
    # x: a Vector of floats
    # y: a Vector of floats
    # output: a vector of floats
    val = [None for i in range(y.num_values)]
    
    for num in range(x.num_values):
        val[num] = a * x[num] + y[num]
    
    return Vector(val)

In [8]:
saxpy(3, Vector((1, 3, 5)), Vector((2, 4, 6)))

Vector((5, 13, 21))

Now we're going to implement talk about what a matrix is, then we're going to make a matrix class!
A matrix is an array of arrays or a collection of vectors.
We just have to note that these arrays should have the same length or if we think of them as a collection of vectors, these vectors have to have the same dimension.

Here's an example of a matrix:
$$
\begin{pmatrix}
1 & 2 & 3 \\
4 & 5 & 6
\end{pmatrix}
$$
or
$$
\begin{pmatrix}
1 & 0 & 0 \\
0 & 1 & 0 \\
0 & 0 & 1
\end{pmatrix}
$$

We denote a matrix by $n \times m$, where $n$ is the number of rows and $m$ is the number of columns.

Another way to write a matrix is
$$
\textbf{A} = (a_{ij})_{ij}.
$$
This tells us the $i$th row and the $j$th column is the value $a_{ij}$.
It's a useful method of representing the matrix abstractly.

Another thing to know about is the transpose of the matrix.
This will be useful shortly.
If
$$
\textbf{A} = (a_{ij})_{ij}.
$$
then the transpose of $\textbf{A}$ is denoted by $\textbf{A}^{\intercal}$ and
$$
\textbf{A}^{\intercal} = (a_{ji})_{ij}.
$$
The columns become the rows and the rows become the columns.

Let's make a matrix class!

In [9]:
class Matrix():
    
    def __init__(self, values):
        # We need to define several class variables, rows, columns, and the matrix values.\
        self.rows = len(values)
        self.cols = len(values[0])
        self.values = self._create_matrix(values)
        
    def _create_matrix(self, values):
        num_cols = len(values[0])
        num_rows = len(values)
        
        for num in range(num_rows):
            if len(values[num]) != num_cols:
                raise Exception(f"Dimension mismatch: {len(num)} != {num_cols}")
        
        for num in range(num_rows):
            values[num] = list(values[num])
            
        return list(values)
        
    def __repr__(self):
        # This needs to return a string that tells you the class and some values.
        # Another way to think about it is that you need to give someone the right amount of
        # information to understand this class.
        return f"Matrix({self.values})"

    def __getitem__(self, i, j=None):
        # i is the row you want to access and j is the column you want to access.
        # Reference the vector class above.
        return self.values[i][j] if j else self.values[i]

    def __len__(self):
        # This should return an integer.
        return len(self.values)
    
    def __str__(self):
        # This should return a string that makes the elements easy to read.
        matrix_string = '['
        for rows in range(self.rows):
            matrix_string += '['
            for cols in range(self.cols):
                matrix_string += str(self.values[rows][cols]) + ', '
            matrix_string = matrix_string[:-2] + ']\n '
        matrix_string = matrix_string[:-2] + ']'
        return matrix_string

    def transpose(self):       
        # Initialize
        transpose = [[0 for i in range(self.rows)] for i in range(self.cols)]

        # Set the values
        for row in range(self.rows):
            for col in range(self.cols):
                transpose[col][row] = self.values[row][col]

        return Matrix(transpose)
    
    def __add__(self, other):
        if not(self.rows == other.rows and self.cols == other.cols):
            raise Exception("The rows or columns do not match.")
        
        add = [[None for i in range(self.cols)] for j in range(self.rows)]
        
        for row in range(self.rows):
            for col in range(self.cols):
                add[row][col] = self.values[row][col] + other.values[row][col]
                
        return Matrix(add)

In [10]:
x = Matrix([[1, 2, 3], [4, 5, 6]])
y = Matrix([[2, 4, 6], [8, 10, 12]])
print(x)

print(x + y)

x[0]

[[1, 2, 3]
 [4, 5, 6]]
[[3, 6, 9]
 [12, 15, 18]]


[1, 2, 3]

We're going to talk about matrix vector multiplications.

**Matrix Vector Multiplication**

If our input is an $n \times m$ matrix $\textbf{A} = (a_{ij})_{ij}$ and an $m$-dimensional vector $\textbf{x} = (x_{1}, \ldots, x_{m})$, then our output is an $n$-dimensional vector.
$$
\textbf{A} \cdot \textbf{x} = \left(\sum_{i=1}^{m} a_{i1}x_{i}, \ldots, \sum_{i=1}^{m} a_{in}x_{i}\right)
$$

In [27]:
def mat_vec_mul(matrix, vector):
    # Our input is a matrix and a vector
    # Our output is a vector
    out = [0 for i in range(matrix.rows)]

    for row in range(matrix.rows):
        print(transpose[row])
        out[row] = dot_product(Vector(matrix[row]), vector)
        
#        for col in range(matrix.cols):
#            out[row] += matrix[row][col] * vector[col]

    return Vector(out)

In [28]:
x = Matrix([[1, 2, 3], [4, 5, 6]])
y = Vector([1, 2, 3])


# [14, 32]
mat_vec_mul(x, y)

[[1, 4]
 [2, 5]
 [3, 6]]
[1, 4]
[2, 5]


Vector((14, 32))

Now, we're going to talk about matrix multiplication.

**Matrix Multiplication**

If we have two matrices, $\textbf{A} = (a_{ij})_{ij}$, an $n \times m$ matrix, and $\textbf{B} = (b_{ij})_{ij}$, an $m \times p$ matrix. 
Note, we cannot multiply matrices unless the number of columns in the first matrix is the same as the number of rows in the second matrix!
The output is a matrix of size $n \times p$ and the values of the matrix are:
$$
\textbf{A} \cdot \textbf{B} = (\sum_{k = 1}^{m} a_{ik}b_{kj})_{ij}
$$

Another way to think about it is that matrix multiplication comes from the dot products of the rows of the first matrix with the columns of the second matrix.
The dot product method is why we implemented the transpose method of the `Matrix` class.

We can also do matrix multiplication with a series of matrix vector operations.
The columns of the new matrix come from the multiplication of the matrix $\textbf{A}$ with the column vectors.

So, you have a number of different ways to think about matrix multiplication and implement it!

In [88]:
def matrix_mult(matrix1, matrix2):
    # The inputs are two matrices.
    # The output is a matrix.
    if matrix1.cols != matrix2.rows:
        raise Exception(f"Dimension mismatch! Rows should equal columns, but {matrix1.cols} != {matrix2.rows}")
    
    out = [[0 for i in range(matrix2.cols)] for j in range(matrix1.rows)]
    
    for row in range(matrix1.rows):
        for col in range(matrix2.cols):
            for k in range(matrix1.cols):
                out[row][col] += matrix1[row][k] * matrix2[k][col]
    
    return Matrix(out)

In [89]:
A = Matrix([[1, 2, 3], [4, 5, 6]])
B = Matrix([[1, 2], [3, 4], [5, 6]])

C = matrix_mult(A, B)
print(C)

[[22, 28]
 [49, 64]]


We're going to talk about the numpy and pytorch libraries and if we have time, we're going to go back to the BLAS functions!

The numpy `ndarray` class and the pytorch `Tensor` class operate similarly.
We're going to just call these tensors from now on.
How we can think of these is that they are an array of an array of an array of arrays, etc.
So, a vector is an array of numbers.
A matrix is an array of an array of numbers.
We can have an array of matrices, which is an array of an array of an array of numbers.
We can keep going.

The **shape** of a tensor is the dimensions.
For an $n$-dimensional vector, it has shape $(n, )$.
For an $n\times m$ matrix, it has shape $(n, m)$.
For a collection of $p$ $n\times m$ matrices would have shape $(p, n, m)$.

The primary example we're going to have is images.
A black and white image can be represented by a matrix, where the values of the matrix correspond to pixels.
So, if we had an image of height 224 pixels and width 224 pixels, the matrix would be a $224 \times 224$ matrix and have shape $(224, 224)$.
Typically, the height is the number of rows and the width is the number of columns in the matrix.

A color image can be represented by 3 matrices, **rgb**, which stands for red green blue.
There is a matrix that corresponds to the red values, a matrix that corresponds to the blue values, and a matrix that corresponds to the green values.
Each of these matrices would have shape $(224, 224)$ and the collection of the matrices would have shape $(3, 224, 224)$.
Typically, we call the number 3 above the **channels**.

If we have a collection of color images, say 16, then that could be represented as a tensor of shape $(16, 3, 224, 224)$.

There are several ways to make an `ndarray` or `tensor`.

In [188]:
x = np.array([[1, 2, 3], [4, 5, 6]])

In [189]:
x

array([[1, 2, 3],
       [4, 5, 6]])

In [190]:
x_tensor = torch.tensor([[1, 2, 3], [4, 5, 6]])

In [191]:
x_tensor

tensor([[1, 2, 3],
        [4, 5, 6]])

In [192]:
x_tensor = torch.tensor(x)

In [193]:
x_tensor

tensor([[1, 2, 3],
        [4, 5, 6]])

In [194]:
x_numpy = x_tensor.numpy()

In [195]:
x_numpy

array([[1, 2, 3],
       [4, 5, 6]])

We can get the shape from the `ndarray` and `tensor` by:

In [196]:
print(x_numpy.shape)
print(x_tensor.shape)

(2, 3)
torch.Size([2, 3])


An important thing to know is that all `ndarrays` and `tensors` are stored as an array of shape $(n, )$.
If we have a `tensor` of shape $(16, 3, 224, 224)$, then it's stored as an array of $16 \times 3 \times 224 \times 224 = 2,408,448$ values.
How the `tensor` transforms this data is by what are called **strides**.
The stride tells us how to skip over the data.

In [199]:
x_numpy.strides

(24, 8)

The layout of `x_numpy` is $[1, 2, 3, 4, 5, 6]$.
Each of these values in the array occupy 8 bytes.
If we want to get element $4$, we would have to go $24 + 8$ bytes into our array.

It should be noted that the numpy strides are based on bytes!
In numpy, ints are 64 bit by default, so they cover 8 bytes.

In [206]:
type(x_numpy[0][0])

numpy.int64

I prefer the way pytorch gives us the strides.
It abstracts away the size of our objects in memory and just treats it like an array of arrays.

In [208]:
x_tensor.stride()

(3, 1)

The layout of `x_tensor` is the same as `x_numpy`.
If we want to access the element in row $i$ and column $j$, we select the $3 * i + j$th element in our array.

If we have a tensor of shape $(16, 3, 28, 28)$, a collection of 16 color images with height and width of 28 pixels.
The stride will be $(3 \times 28 \times 28, 28 \times 28, 28, 1) = (2352, 784, 28, 1)$.
So, if we want to access the pixel at (5, 1, 15, 13), we call `images[4][0][14][12]`.
Under the hood, the tensor is calling `images[4 * 2352 + 0 * 784 + 28 * 14 + 1 * 12]` or `images[9812]`.

Now, we're going to look at how to do our operations above.

In [218]:
tensor1 = torch.tensor([[1, 2, 3], [4, 5, 6]])
tensor2 = torch.tensor([[1, 2], [3, 4], [5, 6]])

In [219]:
torch.matmul(tensor1, tensor2)

tensor([[22, 28],
        [49, 64]])

If we want to do matrix vector multiplication we do:

In [220]:
matrix = torch.tensor([[1, 2, 3], [4, 5, 6], [7, 8, 9]])
vector = torch.tensor([1, 5, 3])

In [221]:
torch.mv(matrix, vector)

tensor([20, 47, 74])

The dot product is:

In [222]:
vector1 = torch.tensor([1, 5, 3])
vector2 = torch.tensor([2, 1, 2])

In [223]:
torch.dot(vector1, vector2)

tensor(13)

Finally, one important concept to know about is broadcasting.
Broadcasting allows us to add or multiply a matrix and a vector together or in general any `tensor` with any `tensor` as long as a few rules are followed:

**Broadcasting rules**

These rules are determined by the shape:
the rightmost elements of the shape are the same or one of them is 1, then proceeds to the left.

So, we can add a $(28, )$-tensor to a $(16, 3, 28, 28)$-tensor.
We can add a $(3, 1, 28)$-tensor to a $(16, 3, 28, 28)$-tensor.
We can add a $(3, 28, 28)$-tensor to a $(16, 3, 28, 28)$-tensor.
Or even a $(3, 4, 1)$-tensor to a $(3, 1, 5)$-tensor.

In [241]:
def is_broadcastable(shape1, shape2):
    
    # Initialization
    broadcast = False
    
    # We need to only check over the smallest shape.
    min_length = min(len(shape1), len(shape2))
    
    # We start at the end.
    shape1 = list(reversed(shape1))
    shape2 = list(reversed(shape2))
    
    # We only keep the necessary elements.
    shape1 = shape1[:min_length]
    shape2 = shape2[:min_length]

    # zip combines shape1 and shape2
    for x, y in zip(shape1, shape2):
        # We check that the shapes are the same or one of them is 1.
        if (x == y) or (x == 1) or (y == 1):
            broadcast = True
        else:
            # If the above condition ever fails, the tensors are not broadcastable.
            return False
    
    return broadcast

In [242]:
is_broadcastable((8, 1, 6, 1), (7, 1, 5))

True

As an exercise, try to implement the following:

In [243]:
def gaxpy(a, b, mat, x_vector, y_vector):
    # Given two scalars a and b, a matrix mat, and two vectors x_vector and y_vector
    # return a vector the same size as y_vector that has the value
    # a * A * x + b * y
    pass

In [245]:
def level3_blas(a, b, A, B, C):
    # Given two scalars a and b and three matrices, A, B, C
    # return a mtrix the same size as C that has the value
    # a * A * B + b * C
    pass